In [ ]:
from exod.processing.bayesian_computations import precompute_bayes_1000, load_precomputed_bayes1000, \
    load_precomputed_bayes_limits, precompute_bayes_limits, get_cube_masks_peak_and_eclipse, B_peak, B_eclipse, \
    PrecomputeBayesLimits
from exod.utils.plotting import cmap_image
from exod.utils.logger import logger
from exod.pre_processing.data_loader import DataLoader
from exod.utils.util import save_df, save_info, get_unique_xy
from exod.xmm.event_list import EventList
from exod.xmm.observation import Observation
from exod.processing.template_based_background_inference import compute_expected_cube_using_templates
from exod.processing.coordinates import get_regions_sky_position, calc_df_regions
from exod.processing.template_based_background_inference import mask_known_sources

from random import shuffle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.colors import ListedColormap, BoundaryNorm
import pandas as pd
from astropy.visualization import ImageNormalize, SqrtStretch
from cv2 import inpaint, INPAINT_NS, INPAINT_TELEA

from exod.pre_processing.data_loader import DataLoader
from exod.xmm.event_list import EventList
from exod.xmm.observation import Observation
from exod.utils.path import data_processed, data_raw, read_observation_ids
from exod.pre_processing.read_events import get_PN_image_file
from exod.utils.logger import logger
from exod.utils.plotting import plot_image, compare_images

import numpy as np
import matplotlib.pyplot as plt
from cv2 import inpaint, INPAINT_NS, INPAINT_TELEA
from skimage.draw import disk
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy.wcs import WCS
from astropy.io import fits
from astropy.convolution import convolve, Gaussian2DKernel
from scipy.interpolate import interp1d

In [ ]:
obsid='0803990401'
size_arcsec=20
time_interval=50
gti_only=False
remove_partial_ccd_frames=True
gti_threshold=1.5
min_energy=0.2
max_energy=10.0
clobber=False
precomputed_bayes_limit=PrecomputeBayesLimits(threshold_sigma=3)

In [ ]:
precomputed_bayes_limit.load()
observation = Observation(obsid)
observation.filter_events(clobber=clobber)
observation.create_images(clobber=clobber)
observation.get_files()
observation.get_events_overlapping_subsets()

for i_subset, subset_overlapping_exposures in enumerate(observation.events_overlapping_subsets):
    savedir = observation.path_results / f'subset_{i_subset}'
    savedir.mkdir(exist_ok=True)
    
    event_list = EventList.from_event_lists(subset_overlapping_exposures)
    dl = DataLoader(event_list=event_list, time_interval=time_interval, size_arcsec=size_arcsec,
                    gti_only=gti_only, min_energy=min_energy, max_energy=max_energy,
                    gti_threshold=gti_threshold, remove_partial_ccd_frames=remove_partial_ccd_frames)
    dl.run()

    img = observation.images[0]
    img.read(wcs_only=True)

    cube_n = dl.data_cube
    cube_mu = compute_expected_cube_using_templates(cube_n, wcs=img.wcs)